<a href="https://colab.research.google.com/github/aarsanjani/applied-machine-learning/blob/master/Team_Virgo_Cluster_Alternus_Vera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Team Virgo Cluster
### Contribution
- Sy Le (006088940) : data import and wrangling, tokenization, remove stopwords, remove stemmings
- Mojdeh Keykhanzadeh(008129589) : remove punctuation , apply ngrams,researched about IBM Faireness
- Hyunwook Shin (012507417) : Build word2vec model for converting raw tokens to structured data, Initial List of Factors 
- Lin Cheng (012484459) : Use TF-IDF Vectorizer + SVD to produce a matrix, and apply Logistic Regression on it
- Yu Xu (012502048): Explored ways to gather topics from the dataset. Explored tf-idf ranking. Explored pipeline + GridSearch for the best n_component of LDA for logistic regression

# Homework: Alternus Vera

Due Tuesday by 11:59pm

Available Oct 30 at 12am - Nov 13 at 11:59pm 15 days

- 1 Read the Article on Fake News (Links to an external site.)Links to an external site. I posted on slack during week 8 lecture. (see 2d below) https://arxiv.org/pdf/1708.01967.pdf
- 2 Project Alternus Vera:
- 2a Use the Kaggle Fake News Data Set,
- 2b Use the "Liar, Liar" DataSet from Politifact.com , that I gave access via Google Drive: https://drive.google.com/open?id=1y3yYF5HHPhH7SyaPwPU9H5HApJzevsFK (Links to an external site.)Links to an external site.
- 2c Use a data source for enriching your previous project with textual data, such as news, reviews, comments, twitter feeds, etc.
- 2d Define factors in a polynomial equation for factors constituting "Fake News/ Factual News": e.g., Reliable Source, Political Affiliation, Sensationalism, Echo Chamber, (see other factors you can derive from paper 1. above)
- 2e Create 3 classification models that assess at least three of these factors and create corresponding weights for your polynomial equation for coming up with a Fake News score.
- 2f Conduct LDA on the three DataSets above and visualize. Interpret results in a Data Narrative; e.g. what are the topics in those data sets?
- 2g Conduct some form of regression analysis (I will leave this open for discussion and your innovation)

Interpret results in a Data Narrative.

## Examples:
- 2g1 Given news can we predict what a politician will say?
- 2g2 Given a news story, break it into its constituent fake/alternative parts

Diagram for Project: https://drive.google.com/open?id=19Xi4zFMMgiCKiKNiqIj_uE_OYEwWoAxQ (Links to an external site.)Links to an external site.

Alternative View: https://drive.google.com/open?id=10UdEYplQj5UYf0X0l_i9eYHY0xoIy8nv (Links to an external site.)Links to an external site.



In [0]:
# dependencies
import pandas as pd
import nltk

# download nltk stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
import pandas as pd
import io
import requests


def get_parsed_data(url):
  return pd.read_csv(io.StringIO(requests.get(url).content.decode('utf-8')), sep='\t', header=None)

def get_parsed_data2(url):
  return pd.read_csv(io.StringIO(requests.get(url).content.decode('utf-8')), sep=',', header='infer')


columns_politifacts = [
  'id',
  'label',
  'statement',
  'subject',
  'speaker',
  'speakers_job_title',
  'state_info',
  'party_affiliation',
  'barely_true',
  'false',
  'half_true',
  'mostly_true',
  'pants_on_fire',
  'context',
]


# download and parse the dataset...
data_poli_test  = get_parsed_data('https://raw.githubusercontent.com/synle/machine-learning-sample-dataset/master/liar_dataset/test.tsv')
data_poli_train = get_parsed_data('https://raw.githubusercontent.com/synle/machine-learning-sample-dataset/master/liar_dataset/train.tsv')
data_poli_valid = get_parsed_data('https://raw.githubusercontent.com/synle/machine-learning-sample-dataset/master/liar_dataset/valid.tsv')
data_kg_fake_news = get_parsed_data2('https://github.com/synle/machine-learning-sample-dataset/raw/master/liar_dataset/kaggle/kaggle-fake.csv')

# parsed the columns
data_poli_test.columns  = columns_politifacts
data_poli_train.columns = columns_politifacts
data_poli_valid.columns = columns_politifacts

In [0]:
data_poli_test.head()

,id,label,statement,subject,speaker,speakers_job_title,state_info,party_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference
2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video


In [0]:
data_kg_fake_news.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [0]:
# Loop Helper for data wrangling
def wrangle_text_loop(fun):
  global data_poli_test
  global data_poli_train
  global data_poli_valid
  global data_kg_fake_news
  
  data_poli_test['statement_clean_tokenize'] = data_poli_test['statement_clean_tokenize'].apply(lambda x: fun(x))
  data_poli_train['statement_clean_tokenize'] = data_poli_train['statement_clean_tokenize'].apply(lambda x: fun(x))
  data_poli_valid['statement_clean_tokenize'] = data_poli_valid['statement_clean_tokenize'].apply(lambda x: fun(x))
  data_kg_fake_news['title_clean_tokenize'] = data_kg_fake_news['title_clean_tokenize'].apply(lambda x: fun(x))
  
  
# this is the first setup  
data_poli_test['statement_clean_tokenize'] = data_poli_test['statement']
data_poli_train['statement_clean_tokenize'] = data_poli_train['statement']
data_poli_valid['statement_clean_tokenize'] = data_poli_valid['statement']
data_kg_fake_news['title_clean_tokenize'] = data_kg_fake_news['title']
  

<h2> Identifying Factors</h2>

| Factor | Rationale | Owner |
| -- | -- | -- | 
| Context  | Type  of the medium (e.g. Large media outlet vs. small blog)  | sy|
| Coverage of topic(s) near date published | Coverage of the latent event from outside sources | hyunwook |
| Political Spectrum | number from 0 to 10 where 0 is left leaning and 10 is right leaning | hyunwook |
| Sensationalism | Ranking from 0 to 10 with 10 being most sensational |  Mojdeh |
| Social Reactions | Ranking from 0 to 10 with 10 being most active in the social interaction | sy|
|Context Consistency | Ranking from 0 to 10 with 10 being most consistent in the topic and statements| Mojdeh |
| WordToVec raw factors | Numeric vector representation of words in doc texts | Yu |
|Text Raw factors|Raw factors(words) mesured by TF-IDF socre| Lin |

## Tokenize The Strings

In [0]:
# tokenize the word
import re

def tokenize(text):
    tokens = re.split('\W+', str(text).lower())
    return tokens
  
wrangle_text_loop(tokenize)

## Remove stopwords

Remove words that are like `we`, `you`, etc...

In [0]:
from nltk.corpus import stopwords

from string import punctuation
# english_stopwords = stopwords.words('english')
english_stopwords = set(stopwords.words('english') + list(punctuation) + [''])


def remove_stopwords(tokenized_words):
  text = [word for word in tokenized_words if word not in english_stopwords]
  return text


wrangle_text_loop(tokenize)

## Stemming and Lemmatizing
Simplify `meaning`, `meant`, `mean` into the same root word `mean`

In [0]:
import nltk


wn = nltk.WordNetLemmatizer()
def lemmatize(tokenized_words):
  text = [wn.lemmatize(word) for word in tokenized_words]
  return text


ps = nltk.PorterStemmer()
def stemming(tokenized_words):
  text = [ps.stem(word) for word in tokenized_words]
  return text

wrangle_text_loop(stemming)
wrangle_text_loop(lemmatize)

In [0]:
data_poli_test.head()

,id,label,statement,subject,speaker,speakers_job_title,state_info,party_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context,statement_clean_tokenize
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview,"[, build, a, wall, on, the, u, s, mexico, bord..."
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,"[, wisconsin, is, on, pace, to, doubl, the, nu..."
2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,"[, say, john, mccain, ha, done, noth, to, help..."
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,"[, suzann, bonamici, support, a, plan, that, w..."
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,"[, when, ask, by, a, report, whether, he, at, ..."


In [0]:
# why '' still there even if '' should have been removed?
data_poli_test.statement_clean_tokenize[0]

['',
 'build',
 'a',
 'wall',
 'on',
 'the',
 'u',
 's',
 'mexico',
 'border',
 'will',
 'take',
 'liter',
 'year',
 '']

In [0]:
data_poli_test['statement_clean_concat'] = data_poli_test.statement_clean_tokenize.map(lambda x: " ".join(x))

In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', stop_words = english_stopwords, sublinear_tf=True)

In [0]:
# calculate tf_idf for each entry
tf_idf_cleaned_statement = vectorizer.fit_transform(data_poli_test['statement_clean_concat'])

In [0]:
tfidf_scores = []
feature_names = vectorizer.get_feature_names()
for i in range(len(data_poli_test)):
  feature_index = tf_idf_cleaned_statement[i,:].nonzero()[1]
  temp = zip(feature_index, [tf_idf_cleaned_statement[i, x] for x in feature_index])
  tfidf_scores.extend([(feature_names[j], s) for (j, s) in temp])

In [0]:
import operator
tfidf_scores.sort(key=operator.itemgetter(1), reverse=True)

## Now we can see the tf-idf scores for the dataset from high to low. Pick the top 100. This ranking can help us to build fake news scoring models.

In [0]:
tfidf_scores[:100]

[('redistrict', 1.0),
 ('debat', 1.0),
 ('grow', 0.772497278191798),
 ('earmark', 0.7687601047099663),
 ('swing', 0.7529560010431467),
 ('appoint', 0.7426905708880303),
 ('toll', 0.7331823895171675),
 ('offshor', 0.7286239569164302),
 ('core', 0.7188739267100904),
 ('socialist', 0.7128796422439648),
 ('exception', 0.7061378693568888),
 ('bush', 0.7028556892532549),
 ('implement', 0.699925913826752),
 ('know', 0.6957909609698617),
 ('common', 0.6951404732112894),
 ('drill', 0.6849139576672708),
 ('financ', 0.6807599255930651),
 ('road', 0.6800320460845184),
 ('unauthor', 0.6731452286939492),
 ('recess', 0.6696347630716403),
 ('millionair', 0.6693301064630943),
 ('root', 0.6680757056311563),
 ('sue', 0.6679381377770834),
 ('custodi', 0.6672170390364597),
 ('150', 0.6617108264285471),
 ('athlet', 0.6596624707706761),
 ('ethanol', 0.6594352004136094),
 ('frisk', 0.6584164194641098),
 ('pledg', 0.657512950684523),
 ('wind', 0.6569918495570503),
 ('cap', 0.6563506103143171),
 ('nato', 0.6544

## Remove Punctuation 

In [0]:
import string

def remove_punctuation(tokenized_words):
    table = str.maketrans('', '', string.punctuation)
    text = [w.translate(table) for w in tokenized_words]
    return text 
  
wrangle_text_loop(remove_punctuation)


In [0]:
data_poli_test.head()

,id,label,statement,subject,speaker,speakers_job_title,state_info,party_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context,statement_clean_tokenize,statement_clean_concat
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview,"[, build, a, wall, on, the, u, s, mexico, bord...",build a wall on the u s mexico border will ta...
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,"[, wisconsin, is, on, pace, to, doubl, the, nu...",wisconsin is on pace to doubl the number of l...
2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,"[, say, john, mccain, ha, done, noth, to, help...",say john mccain ha done noth to help the vet
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,"[, suzann, bonamici, support, a, plan, that, w...",suzann bonamici support a plan that will cut ...
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,"[, when, ask, by, a, report, whether, he, at, ...",when ask by a report whether he at the center...


In [0]:
data_kg_fake_news.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,...,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type,title_clean_tokenize
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,...,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,"[, muslim, bust, they, stole, million, in, gov..."
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,...,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,"[, re, whi, did, attorney, gener, loretta, lyn..."
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,...,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,"[, break, weiner, cooper, with, fbi, on, hilla..."
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,...,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias,"[, pin, drop, speech, by, father, of, daughter..."
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,...,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias,"[, fantast, trump, s, 7, point, plan, to, refo..."


## Apply ngrams 

In [0]:
#bigrams 
from nltk import bigrams
from nltk.util import ngrams
def apply_ngrams(tokenized_words):
  bigrams_list =  list (ngrams(tokenized_words,2))
  return bigrams_list

wrangle_text_loop(apply_ngrams)
#data_poli_test.head()

  


## TfidfVectorizer/SVD/Logistic Regression

Now try to use TfidfVectorizer to get a matrix for further classification. To make it easier, we make the clssification binary. We also try applying SVD for dimension reduction.

In [0]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
import numpy as np
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from string import punctuation
from nltk import PorterStemmer
import copy 

nltk.download('punkt')

cachedStopWords = set(stopwords.words('english') + list(punctuation) + [''])

print(data_poli_train.groupby(['label'])['label'].count())

data_poli_train_b = copy.deepcopy(data_poli_train);
data_poli_test_b = copy.deepcopy(data_poli_test);
data_poli_train_b.loc[data_poli_train_b['label'].isin(['barely-true','half-true', 'pants-fire']), 'label'] = 'false'
data_poli_train_b.loc[data_poli_train_b['label'].isin(['mostly-true','true']), 'label'] = 'true'
data_poli_test_b.loc[data_poli_test_b['label'].isin(['barely-true','half-true', 'pants-fire']), 'label'] = 'false'
data_poli_test_b.loc[data_poli_test_b['label'].isin(['mostly-true','true']), 'label'] = 'true'

print(data_poli_train_b.groupby(['label'])['label'].count())

def tokenize2(text):

    min_length = 3
    words = map(lambda word: word.lower(), word_tokenize(text))
    words = [word for word in words if word not in cachedStopWords]
    tokens = (list(map(lambda token: PorterStemmer().stem(token), words)))
    p = re.compile('[a-zA-Z]+')
    filtered_tokens = list(filter(lambda token: p.match(token) and len(token) >= min_length, tokens))
    return filtered_tokens

vectorizer = TfidfVectorizer(tokenizer=tokenize2)

svd_model = TruncatedSVD(n_components=200,       
                         algorithm='randomized',
                         n_iter=10)
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])
# svd_transformer=vectorizer
    
print((data_poli_train_b["statement"]).shape)

vectorised_train_documents = svd_transformer.fit_transform(data_poli_train_b["statement"])
print (vectorised_train_documents.shape)

train_labels = data_poli_train_b["label"]
print (train_labels.shape)

vectorised_test_documents = svd_transformer.transform(data_poli_test_b["statement"])
test_labels = data_poli_test_b["label"]

print(data_poli_test_b.groupby(['label'])['label'].count())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
label
barely-true    1654
false          1995
half-true      2114
mostly-true    1962
pants-fire      839
true           1676
Name: label, dtype: int64
label
false    6602
true     3638
Name: label, dtype: int64
(10240,)
(10240, 200)
(10240,)
label
false    818
true     449
Name: label, dtype: int64


Then apply LogisticRegression on the TF-IDF output:

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.preprocessing import LabelEncoder, Imputer, MaxAbsScaler
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB

scaler = MaxAbsScaler()
vectorised_train_documents = scaler.fit_transform(vectorised_train_documents)

# model = svm.SVC();
# parameters = {'kernel':('linear','poly','rbf','sigmoid')}
# lr_gridCV = GridSearchCV(model, parameters, cv=5, scoring='accuracy')
# lr_gridCV = lr_gridCV.fit(vectorised_train_documents, train_labels)

# print(lr_gridCV.best_score_)
# print(lr_gridCV.best_params_)
# y_pred = lr_gridCV.predict(vectorised_test_documents)
# print("Accuracy:",metrics.accuracy_score(test_labels, y_pred))

logistic = LogisticRegression(class_weight={"false":3,"true":5})
C = [0.01, 0.1, 1, 10]
penalty = ['l1','l2']

param_grid = dict(C=C, penalty=penalty)
gs = GridSearchCV(logistic, param_grid=param_grid, cv= 5, scoring='accuracy')

gs.fit(vectorised_train_documents, train_labels)
print(gs.best_params_)

y_pred=gs.predict(vectorised_test_documents)
print("Accuracy:",metrics.accuracy_score(test_labels, y_pred))
print(pd.DataFrame(metrics.confusion_matrix(test_labels, y_pred),
             index=[['actual', 'actual'], ['true', 'false']],
             columns=[['predicted', 'predicted'], ['true', 'false']]))
# print(pd.Series(
#     gs.best_estimator_.coef_.T.ravel(),
#     index=vectorizer.get_feature_names()
# ).sort_values(ascending=False)[:20])


KeyboardInterrupt: ignored

In [0]:
from sklearn.model_selection import train_test_split

print(data_kg_fake_news.shape)
print(data_kg_fake_news.groupby(['type'])['type'].count())

data_kg_fake_news_b=copy.deepcopy(data_kg_fake_news);
data_kg_fake_news_b.loc[data_kg_fake_news_b['type']!='bs', 'type'] = 'non-bs'
X=data_kg_fake_news_b['text']
y=data_kg_fake_news_b['type']

print(data_kg_fake_news_b.groupby(['type'])['type'].count())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

vectorizer = TfidfVectorizer(tokenizer=tokenize2)
svd_model = TruncatedSVD(n_components=200,       
                         algorithm='randomized',
                         n_iter=10)
# svd_transformer = Pipeline([('tfidf', vectorizer), 
#                             ('svd', svd_model)])
svd_transformer=vectorizer
vectorised_train_documents2 = svd_transformer.fit_transform(X_train.astype('U'))
vectorised_test_documents2 = svd_transformer.transform(X_test.astype('U'))


(12999, 21)
type
bias            443
bs            11492
conspiracy      430
fake             19
hate            246
junksci         102
satire          146
state           121
Name: type, dtype: int64


In [0]:
from sklearn.ensemble import RandomForestClassifier

# logistic = LogisticRegression(class_weight={"bs":1,"non-bs":5})
# C = [0.1, 1]
# penalty = ['l2']

# param_grid = dict(C=C, penalty=penalty)
# gs = GridSearchCV(logistic, param_grid=param_grid, cv= 5, scoring='accuracy')
# gs.fit(vectorised_train_documents2, y_train)

gs=RandomForestClassifier()
gs.fit(vectorised_train_documents2, y_train)
print(vectorised_train_documents2.shape)
feature_imp = pd.Series(gs.feature_importances_,index=list(vectorizer.get_feature_names())).sort_values(ascending=False).nlargest(20)
print(feature_imp)

y_pred=gs.predict(vectorised_test_documents2)
print(y_test.value_counts(sort=False))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))


(9099, 97177)
nan           0.028139
naturalnew    0.008137
disqu         0.004675
min           0.004579
via           0.004064
load          0.003847
greenfield    0.003593
trump         0.003074
believ        0.003066
sjw           0.003061
infowar       0.002848
women         0.002602
afp           0.002393
keyword       0.002185
duke          0.002108
patrick       0.002057
becom         0.001851
toward        0.001726
part          0.001717
easley        0.001685
dtype: float64
bs        3402
non-bs     498
Name: type, dtype: int64
Accuracy: 0.8902564102564102
[[3394    8]
 [ 420   78]]


In [0]:
data1 = data_kg_fake_news_b.loc[data_kg_fake_news_b['type']=='bs', :]
for col in ['domain_rank','spam_score','replies_count','participants_count', 'likes', 'comments', 'shares', 'type']:
  print(col, ":", data1[col].unique())

domain_rank : [   nan 18533. 78393. 25842.  3734. 94243. 42160. 12993. 48959. 61606.
  6659.  7587. 93026.   967. 22665. 10352. 87365.   486. 23950. 39980.
 36674. 60570. 77628. 22680. 25709.   553. 48175. 35381. 32069. 40299.
 40480. 12387. 70635. 51784. 56000. 78345. 60463. 17592. 67400. 65917.
 82758. 68648. 32398. 79662. 53149. 26413. 30230. 23040. 21114. 68691.
 98679. 54210. 43327. 27046. 23349. 10414. 61423. 91187. 12191. 75353.
 17423. 22432. 44451. 57497. 28149.   538. 31016. 63847. 62759. 22891.
 91245.  1616. 96853. 18896. 91878. 33221. 63072. 56487. 65975. 65060.
  8968. 18369. 93150.  2182. 19375. 42575. 37614.  9645. 14958. 30890.
 84363. 24453. 49688. 23405. 72287. 14170. 51246.  3921. 85288. 34478.
 62087. 48413. 48717. 64783. 58305. 46814. 36845. 14716. 17366. 67959.
 35234. 43665.  2435.]
spam_score : [0.    0.038 0.061 0.018 0.007 0.002 0.009 0.141 0.004 0.001 0.003 0.005
 0.04  0.01  0.995 0.131 0.013 0.928 0.043 0.334 0.008 0.05  0.101 0.028
 0.105 0.71  0.842 0.09

In [0]:
data1.text

59       United States Marine Field McConnell  Plum Cit...
60       So ,you have Rothschild banksters and British ...
61       Here is the problem . The USA constitution sta...
62       There is plenty of proof the machines are rigg...
63       Trump has an excuse now to audit any vote with...
64       He has got to go after him , he is the one cau...
65       He has got to go after him , he is the one cau...
66       There is plenty of proof the machines are rigg...
67       Trump has an excuse now to audit any vote with...
68       He has got to go after him , he is the one cau...
69       United States Marine Field McConnell  Plum Cit...
70       Field is correct about the 8a companies and Tr...
71       Georg Soros the good oil . http://mailstar.net...
72       There is a lot more than meets the eye to this...
73       They tick all the boxes , flying under the rad...
74       Field is correct about the 8a companies and Tr...
75       Georg Soros the good oil . http://mailstar.net.

<h2>Political Spectrum</h2>

<h2>Word2Vec</h2>
(hshin)

Create a word2vector model. This model will be useful identifying the political affiliations of the document. For each non-trivial words (after distiliation), we create a similarity score with the words that are commonly used by authors associated wiht one party or another (such as "tax cuts", "spending" with republican and "bigotry", "impeach" with republicans as an example). It is important to note that the words frequently chosen by one party may not necessarily be something they support. For example, authors to the right of the spectrum do not like spending. Hence,  it is more likely that they would talk about spending than the democrats.

If the document associate closely with the dictionary of words with specific political affiliations you can set the poliyical affiliation score accordingly. The higher the similarity, the greater the weight. 

---



In [0]:
!pip install gensim

    100% |████████████████████████████████| 23.6MB 956kB/s 
    100% |████████████████████████████████| 1.4MB 13.7MB/s 
    100% |████████████████████████████████| 133kB 28.1MB/s 
    100% |████████████████████████████████| 61kB 17.8MB/s 
    100% |████████████████████████████████| 4.8MB 6.8MB/s 
    100% |████████████████████████████████| 552kB 24.0MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
from gensim.models import Word2Vec


statements = data_poli_train['statement_clean_tokenize']
statements.head()

0    [, say, the, annies, list, political, group, s...
1    [, when, did, the, decline, of, coal, start, i...
2    [, hillary, clinton, agrees, with, john, mccai...
3    [, health, care, reform, legislation, is, like...
4    [, the, economic, turnaround, started, at, the...
Name: statement_clean_tokenize, dtype: object

In [0]:
m = Word2Vec( statements )

m.similarity( "plan", "cut")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.782147

In [0]:
m.similarity( "job", "economy" )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.7834891

In [0]:
m.similarity( "people", "vote" )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.6091816

In [0]:
# use pipe and gridsearch to find which n_components is the best for LDA and the following logistic regression
pipe = Pipeline([('tfidf', vectorizer), 
                 ('svd', svd_model),
                 ('scaler', scaler),
                 ('logistic', logistic)
                ])

X_train_, y_train_, X_test_, y_test_ = data_poli_train_b["statement"].tolist(),train_labels,data_poli_test_b["statement"].tolist(), test_labels

parameters = {'svd__n_components': [200,500,1000],'logistic__C':C}
grid_tf_idf_ = GridSearchCV(pipe, param_grid=parameters, cv=5, scoring='accuracy')
grid_tf_idf_.fit(X_train_, y_train_)

In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2018-11-16 00:15:48--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-11-16 00:15:48--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  1.21MB/s    in 15m 59s 

2018-11-16 00:31:47 (878 KB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
import numpy as np

with open("glove.6B.50d.txt", "rb") as lines:
    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [0]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD

etree_w2v = Pipeline([
    ("word2vec_vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("SVD", TruncatedSVD(n_components = 200)),
    ("random_forest", RandomForestClassifier())])

TypeError: ignored